# Clinical Heart Failure Detection Using Whole-Slide Images of H&E tissue

**Version**
- **0.04**: Class Label Info in List was not accurate - fixed it. Now, Train/Validate/Test Accuracy is more realistic = ~55%
- **0.03**: Tweak CNN model done for MNIST to work for this dataset to have a working end to end CNN model. Train/Validate/Test Accuracy = ~100%
- **0.02**: Prepare Train/Validate/Test Labels and Images 
- **0.01**: Prepare Train/Validate/Test Images

**Improvement Opportunity**
- Convert code sections in data preparation for train/validation/test to functions
- As this is a 2 class classification - loss function can be changed to binary_crossentropy instead of categorical_crossentropy
- Reduce parameters, epochs.

## Download Dataset

### Download Train, Validate and Test Images
- Source Link to the Dataset / Annotation File: https://idr.openmicroscopy.org/webclient/?show=project-402
- Follow the instructions at following link, install IBM Aspera Desktop Client to download the dataset.
- Copy downloaded folders to '**data/images**' folder in your working directory where you have this Jupyter Notebook:
  - 'held-out_validation'
  - 'training'

### Download Label Information for Train, Validate and Test Images 
- Following link will point to below Github link which has the annotation File: https://idr.openmicroscopy.org/webclient/?show=project-402
- Source Link for the Annotation File: https://github.com/IDR/idr0042-nirschl-wsideeplearning/tree/master/experimentA
- Download and copy file '**idr0042-experimentA-annotation.csv**' to '**data/labels/**' folder in your working directory where you have this Jupyter Notebook

## References 
#### Data Preparation
- Reading an image
  - mathplotlib: https://stackoverflow.com/questions/9298665/cannot-import-scipy-misc-imread
  - pathlib: https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f#:~:text=To%20use%20it%2C%20you%20just,for%20the%20current%20operating%20system.
  - OpenCV: https://www.geeksforgeeks.org/python-opencv-cv2-imread-method/
- Load multiple images into a numpy array
  - glob / os.listdir: https://stackoverflow.com/questions/39195113/how-to-load-multiple-images-in-a-numpy-array
  - glob / cv2: https://medium.com/@muskulpesent/create-numpy-array-of-images-fecb4e514c4b
- Load a CSV file
  - Datacamp: https://www.datacamp.com/community/tutorials/pandas-read-csv?utm_source=adwords_ppc&utm_campaignid=1455363063&utm_adgroupid=65083631748&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=278443377095&utm_targetid=dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=9061994&gclid=EAIaIQobChMIz5TKz-v17QIV1AorCh0bfw96EAAYASAAEgKiGPD_BwE
- Split a String
  - Python Central: https://www.pythoncentral.io/cutting-and-slicing-strings-in-python/


## Data Preparation

### Understand Images Folder Structure and Number of Images Available

**Training/Validation**
- \..\training\fold_1: has images for training = 770#
- \..\training\test_fold_1: has images for validation = 374#
- Total = 770 + 374 = 1144 images

**Test**
- \..\held-out_validation: has images for testing = 1155#

### Understand Annotation File and Label Information Available

Relevant columns of interest:
- Column A: Dataset Name: Classifies each row/instance as 'training' or 'test'
- Column B: Image Name: Specifies filename of the image for the row/instance
- Column Z: Experimental Condition [Diagnosis]: has 3 classes:
  - 'chronic heart failure'
  - 'heart tissue pathology' - We will treat this as 'not chronic heart failure'
  - 'not chronic heart failure'
- Column AA: Channels: mentions RGB => images are color images and will have 3 channels Red/Green/Blue (for CNN). 
  
Breakup of training/test instances:
- **training**
  - 'chronic heart failure' = 517
  - 'not chronic heart failure' = 627
- **test**
  - 'chronic heart failure' = 517
  - 'not chronic heart failure' = 638

Total '**training**' = 517 + 627 = 1144
- Note: 'validate' is a portion of this 'training' set.

Total '**test**' = 517 + 638 = 1155

### Load libraries to aid in converting images to arrays

We will convert the images to arrays so that we can then use them to feed to our CNN model.

In [1]:
# install OpenCV package
# pip install opencv-python

In [2]:
import cv2

In [3]:
import glob

In [4]:
import numpy as np

### Load libraries to aid reading CSV file to dataframe

We will import the annotation file into a Pandas Dataframe so that we can then access the labels information.

In [5]:
import pandas as pd

In [6]:
labels = pd.read_csv('data/labels/idr0042-experimentA-annotation.csv')

In [7]:
labels

,Dataset Name,Image Name,Characteristics [Organism],Term Source 1 REF,Term Source 1 Accession,Characteristics [Organism Part],Term Source 2 REF,Term Source 2 Accession,Characteristics [Diagnosis],Term Source 3 REF,...,Characteristics [Ethnic or Racial Group],Term Source 6 REF,Term Source 6 Accession,Characteristics [Age],Characteristics [Individual],Characteristics [Clinical History],Protocol REF,Protocol REF.1,Experimental Condition [Diagnosis],Channels
0,training,33381_0_fal_10_0.png,Homo sapiens,NCBITaxon,NCBITaxon_9606,heart,UBERON,UBERON_0000948,chronic heart failure,SNOMED,...,African American,SNOMED,SNOMED_S-62310,65 years,33381,ischemic cardiomyopathy,treatment protocol,image acquisition,chronic heart failure,RGB
1,training,33381_0_fal_14_0.png,Homo sapiens,NCBITaxon,NCBITaxon_9606,heart,UBERON,UBERON_0000948,chronic heart failure,SNOMED,...,African American,SNOMED,SNOMED_S-62310,65 years,33381,ischemic cardiomyopathy,treatment protocol,image acquisition,chronic heart failure,RGB
2,training,33381_0_fal_16_0.png,Homo sapiens,NCBITaxon,NCBITaxon_9606,heart,UBERON,UBERON_0000948,chronic heart failure,SNOMED,...,African American,SNOMED,SNOMED_S-62310,65 years,33381,ischemic cardiomyopathy,treatment protocol,image acquisition,chronic heart failure,RGB
3,training,33381_0_fal_18_0.png,Homo sapiens,NCBITaxon,NCBITaxon_9606,heart,UBERON,UBERON_0000948,chronic heart failure,SNOMED,...,African American,SNOMED,SNOMED_S-62310,65 years,33381,ischemic cardiomyopathy,treatment protocol,image acquisition,chronic heart failure,RGB
4,training,33381_0_fal_25_0.png,Homo sapiens,NCBITaxon,NCBITaxon_9606,heart,UBERON,UBERON_0000948,chronic heart failure,SNOMED,...,African American,SNOMED,SNOMED_S-62310,65 years,33381,ischemic cardiomyopathy,treatment protocol,image acquisition,chronic heart failure,RGB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2294,test,36175_1_nrm_18_0.png,Homo sapiens,NCBITaxon,NCBITaxon_9606,heart,UBERON,UBERON_0000948,not chronic heart failure,SNOMED,...,Caucasian,SNOMED,SNOMED_S-0003D,53 years,36175,normal cardiovascular function by cardiac cath...,treatment protocol,image acquisition,not chronic heart failure,RGB
2295,test,36175_1_nrm_1_0.png,Homo sapiens,NCBITaxon,NCBITaxon_9606,heart,UBERON,UBERON_0000948,not chronic heart failure,SNOMED,...,Caucasian,SNOMED,SNOMED_S-0003D,53 years,36175,normal cardiovascular function by cardiac cath...,treatment protocol,image acquisition,not chronic heart failure,RGB
2296,test,36175_1_nrm_20_0.png,Homo sapiens,NCBITaxon,NCBITaxon_9606,heart,UBERON,UBERON_0000948,not chronic heart failure,SNOMED,...,Caucasian,SNOMED,SNOMED_S-0003D,53 years,36175,normal cardiovascular function by cardiac cath...,treatment protocol,image acquisition,not chronic heart failure,RGB
2297,test,36175_1_nrm_21_0.png,Homo sapiens,NCBITaxon,NCBITaxon_9606,heart,UBERON,UBERON_0000948,not chronic heart failure,SNOMED,...,Caucasian,SNOMED,SNOMED_S-0003D,53 years,36175,normal cardiovascular function by cardiac cath...,treatment protocol,image acquisition,not chronic heart failure,RGB


In [8]:
print(labels['Dataset Name'])

0       training
1       training
2       training
3       training
4       training
          ...   
2294        test
2295        test
2296        test
2297        test
2298        test
Name: Dataset Name, Length: 2299, dtype: object


In [9]:
print(labels['Dataset Name'][0])

training


In [10]:
type(labels['Dataset Name'][0])

str

In [11]:
print(labels['Image Name'])

0       33381_0_fal_10_0.png
1       33381_0_fal_14_0.png
2       33381_0_fal_16_0.png
3       33381_0_fal_18_0.png
4       33381_0_fal_25_0.png
                ...         
2294    36175_1_nrm_18_0.png
2295     36175_1_nrm_1_0.png
2296    36175_1_nrm_20_0.png
2297    36175_1_nrm_21_0.png
2298     36175_1_nrm_2_0.png
Name: Image Name, Length: 2299, dtype: object


In [12]:
print(labels['Image Name'][0])

33381_0_fal_10_0.png


In [13]:
type(labels['Image Name'][0])

str

In [14]:
print(labels['Experimental Condition [Diagnosis]'])

0           chronic heart failure
1           chronic heart failure
2           chronic heart failure
3           chronic heart failure
4           chronic heart failure
                  ...            
2294    not chronic heart failure
2295    not chronic heart failure
2296    not chronic heart failure
2297    not chronic heart failure
2298    not chronic heart failure
Name: Experimental Condition [Diagnosis], Length: 2299, dtype: object


In [15]:
print(labels['Experimental Condition [Diagnosis]'][0])

chronic heart failure


In [16]:
type(labels['Experimental Condition [Diagnosis]'][0])

str

In [17]:
# confirm 'no info' cells have been encoded as 'nan'... check one entry
print(labels['Characteristics [Disease Subtype]'][463])

nan


### Prepare Train Images and Train Labels

In [18]:
# read filepath for all "filenames with extension as 'png'" into a list
# here filepath means 'relative directory + filename'
filepathlist_train = glob.glob('data/images/training/fold_1/*.png')

In [19]:
# confirm you have got the total number of desired items in the list
len(filepathlist_train)

770

We need to extract label info (for an image) from the labels dataframe by using the filename of the image.

In [20]:
# check what an element in the filelist contain
# it has both directory information and the filename
# we need to split the string to get the filename 
# the filename can then be used to check for the label info in the labels dataframe
filepathlist_train[0]

'data/images/training/fold_1\\33381_0_fal_10_0.png'

In [21]:
# split the string
directory, filename = filepathlist_train[0].split('\\')

In [22]:
# gives the directory info
directory

'data/images/training/fold_1'

In [23]:
# gives the filname we need
filename

'33381_0_fal_10_0.png'

In [24]:
# read a file using the list containing the file path
img = cv2.imread(filepathlist_train[0])

In [25]:
# define the empty list that need to populated with info
train_images = []
train_labels = []
# iterate for all items in the file path list
for filepath in filepathlist_train:
    # prepare image list
    img = cv2.imread(filepath)
    train_images.append(img)
    # prepare labels list
    # extract filename from the file path
    directory, filename = filepath.split('\\')
    # iterate for all items in our labels dataframe to search for the label
    for index in range(len(labels)):
        # we will compare the filename with all the filenames in the 'Image Name' column of the labels dataframe
        # when there is a match, we will copy the label from the 'Experimental Condition [Diagnosis]' column
        if (filename == labels['Image Name'][index]):
            label = labels['Experimental Condition [Diagnosis]'][index]
            # encode Class1 and Class0 as applicable
            if (label == 'chronic heart failure'):
                label = 1
            elif (label == 'not chronic heart failure'):
                label = 0
            elif (label == 'heart tissue pathology'):
                label = 0
            # append the label to the list
            train_labels.append(label) 

In [26]:
# train_labels

Convert images to numpy arrays and confirm shape is as required for CNN. 

In [27]:
# confirm you have got the total number desired images in the list
len(train_images)

770

In [28]:
# train is a list
type(train_images)

list

In [29]:
# convert list to a numpy array and the values to float
train_images = np.array(train_images, dtype = 'float32')

In [30]:
# check the shape to confirm it is ready for CNN
# number of instances, width, height, number of channels
# number of instances = number of image
# number of channels = 3 ... as these are color images
train_images.shape

(770, 250, 250, 3)

Convert labels to numpy arrays and confirm shape is as required for CNN. 

In [31]:
len(train_labels)

770

In [32]:
type(train_labels)

list

In [33]:
train_labels[0]

1

In [34]:
train_labels[432]

0

In [35]:
# convert list to a numpy array and the values to int64
train_labels = np.array(train_labels, dtype = 'int64')

In [36]:
# check the shape to confirm it is ready for CNN
train_labels.shape

(770,)

In [37]:
len(train_labels)

770

Let us check on the number of Class 0 and Class 1s that we have. 

In [38]:
count_ones = 0
count_zeroes = 0
for i in range(len(train_labels)):
    if (train_labels[i] == 1):
            count_ones += 1
    elif(train_labels[i] == 0):
            count_zeroes += 1
print('Total Labels:',(count_ones + count_zeroes))
print('# of Class 1:',count_ones)   
print('# of Class 0:',count_zeroes)    

Total Labels: 770
# of Class 1: 352
# of Class 0: 418


We will convert the labels to 2bit values: 01 and 10 to correspond to the 2 classes. This is required to match to the model's output layer expectation so that we can effectively train and test. 

In [39]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [40]:
# convert labels to categorical
train_labels = to_categorical(train_labels)

### Prepare Validation Images and Validation Labels

In [41]:
# read filepath for all "filenames with extension as 'png'" into a list
# here filepath means 'relative directory + filename'
filepathlist_validation = glob.glob('data/images/training/test_fold_1/*.png')

In [42]:
# define the empty list that need to populated with info
validation_images = []
validation_labels = []
# iterate for all items in the file path list
for filepath in filepathlist_validation:
    # prepare image list
    img = cv2.imread(filepath)
    validation_images.append(img)
    # prepare labels list
    # extract filename from the file path
    directory, filename = filepath.split('\\')
    # iterate for all items in our labels dataframe to search for the label
    for index in range(len(labels)):
        # we will compare the filename with all the filenames in the 'Image Name' column of the labels dataframe
        # when there is a match, we will copy the label from the 'Experimental Condition [Diagnosis]' column
        if (filename == labels['Image Name'][index]):
            label = labels['Experimental Condition [Diagnosis]'][index]
            # encode Class1 and Class0 as applicable
            if (label == 'chronic heart failure'):
                label = 1
            elif (label == 'not chronic heart failure'):
                label = 0
            elif (label == 'heart tissue pathology'):
                label = 0
            # append the label to the list
            validation_labels.append(label) 

In [43]:
# convert list to a numpy array and the values to float
validation_images = np.array(validation_images, dtype = 'float32')

In [44]:
# check the shape to confirm it is ready for CNN
validation_images.shape

(374, 250, 250, 3)

In [45]:
# convert list to a numpy array and the values to int64
validation_labels = np.array(validation_labels, dtype = 'int64')

In [46]:
# check the shape to confirm it is ready for CNN
validation_labels.shape

(374,)

Let us check on the number of Class 0 and Class 1s that we have. 

In [47]:
count_ones = 0
count_zeroes = 0
for i in range(len(validation_labels)):
    if (validation_labels[i] == 1):
            count_ones += 1
    elif(validation_labels[i] == 0):
            count_zeroes += 1
print('Total Labels:',(count_ones + count_zeroes))
print('# of Class 1:',count_ones)   
print('# of Class 0:',count_zeroes)   

Total Labels: 374
# of Class 1: 165
# of Class 0: 209


In [48]:
# convert labels to categorical
validation_labels = to_categorical(validation_labels)

### Prepare Test Images and Test Labels

In [49]:
# read filepath for all "filenames with extension as 'png'" into a list
# here filepath means 'relative directory + filename'
filepathlist_test = glob.glob('data/images/held-out_validation/*.png')

In [50]:
# define the empty list that need to populated with info
test_images = []
test_labels = []
# iterate for all items in the file path list
for filepath in filepathlist_test:
    # prepare image list
    img = cv2.imread(filepath)
    test_images.append(img)
    # prepare labels list
    # extract filename from the file path
    directory, filename = filepath.split('\\')
    # iterate for all items in our labels dataframe to search for the label
    for index in range(len(labels)):
        # we will compare the filename with all the filenames in the 'Image Name' column of the labels dataframe
        # when there is a match, we will copy the label from the 'Experimental Condition [Diagnosis]' column
        if (filename == labels['Image Name'][index]):
            label = labels['Experimental Condition [Diagnosis]'][index]
            # encode Class1 and Class0 as applicable
            if (label == 'chronic heart failure'):
                label = 1
            elif (label == 'not chronic heart failure'):
                label = 0
            elif (label == 'heart tissue pathology'):
                label = 0
            # append the label to the list
            test_labels.append(label) 

In [51]:
# convert list to a numpy array and the values to float
test_images = np.array(test_images, dtype = 'float32')

In [52]:
# check the shape to confirm it is ready for CNN
test_images.shape

(1155, 250, 250, 3)

In [53]:
# convert list to a numpy array and the values to int64
test_labels = np.array(test_labels, dtype = 'int64')

In [54]:
# check the shape to confirm it is ready for CNN
test_labels.shape

(1155,)

Let us check on the number of Class 0 and Class 1s that we have. 

In [55]:
count_ones = 0
count_zeroes = 0
for i in range(len(test_labels)):
    if (test_labels[i] == 1):
            count_ones += 1
    elif(test_labels[i] == 0):
            count_zeroes += 1
print('Total Labels:',(count_ones + count_zeroes))
print('# of Class 1:',count_ones)   
print('# of Class 0:',count_zeroes)   

Total Labels: 1155
# of Class 1: 517
# of Class 0: 638


In [56]:
# convert labels to categorical
test_labels = to_categorical(test_labels)

## Define Model

In [57]:
from keras import models

In [58]:
from keras import layers

In [59]:
model_cnn = models.Sequential()

Layer Details:
- 2 dimensional Convolution Layer
- Number of filters/kernels = 32
- Filter/Kernel Size = 3x3
- Activation Function = relu (for non-linearity detection)
- Input Shape = 250x250 matrix with 3 channel (as we have a color image)

In [60]:
model_cnn.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(250,250,3)))

Layer Details:
- Downsample the output from previous layer
- We will take the max value for a every 2x2 window ... moved over the input

In [61]:
model_cnn.add(layers.MaxPooling2D(2,2))

Layer Details:
- 2 dimensional Convolution Layer
- Number of filters/kernels = 64
- Filter/Kernel Size = 3x3
- Activation Function = relu (for non-linearity detection)

In [62]:
model_cnn.add(layers.Conv2D(64, (3,3), activation = 'relu'))

Layer Details:
- Downsample the output from previous layer
- We will take the max value for a every 2x2 window ... moved over the input

In [63]:
model_cnn.add(layers.MaxPooling2D(2,2))

Layer Details:
- 2 dimensional Convolution Layer
- Number of filters/kernels = 64
- Filter/Kernel Size = 3x3
- Activation Function = relu (for non-linearity detection)

In [64]:
model_cnn.add(layers.Conv2D(64, (3,3), activation='relu'))

Data at this stage is in matrix form. We will convert it to vector form to feed to a fully connected network (FCN).

In [65]:
model_cnn.add(layers.Flatten())

We will design for 64 outputs with activation function as relu (to learn non-linearity).

In [66]:
model_cnn.add(layers.Dense(64, activation = 'relu'))

This is the final layer. Hence, the outputs will be 2 corresponding to the 2 classes:
- clinical heart failure = yes: 1
- clinical heart failure = no: 0

Activation Function chosen here is softmax to have a probabilistic output. 

In [67]:
model_cnn.add(layers.Dense(2, activation = 'softmax'))

In [68]:
model_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 248, 248, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 124, 124, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 122, 122, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 61, 61, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 59, 59, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 222784)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

## Define Optimizer, Loss Function and Metrics to be used for the Model
- Going ahead with the well known functions at this point in time
- Selected accuracy as the metrics to understand validation / test accuracy of the model

In [69]:
model_cnn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

## Train and Validate the Model
#### We will now train the model using train images and train labels. 
- We will use a batch size = 10.
- 1 epoch = 770 / 10 = 77 batches
- 1 epoch = 1 complete run of all train samples for training the model
- We will go for a total of 5 epochs = 5 complete run of the all train samples

#### We will validate the model using validation images and validation labels.

In [70]:
model_cnn.fit(train_images, train_labels, epochs = 5, batch_size = 10, validation_data = (validation_images, validation_labels))

Train on 770 samples, validate on 374 samples
Epoch 1/5
770/770 [==============================] - 71s 92ms/step - loss: 524.3493 - accuracy: 0.5130 - val_loss: 0.6987 - val_accuracy: 0.4759
Epoch 2/5
770/770 [==============================] - 69s 90ms/step - loss: 49.0332 - accuracy: 0.5299 - val_loss: 0.6958 - val_accuracy: 0.4866
Epoch 3/5
770/770 [==============================] - 69s 89ms/step - loss: 2.1133 - accuracy: 0.5675 - val_loss: 0.6910 - val_accuracy: 0.5588
Epoch 4/5
770/770 [==============================] - 68s 88ms/step - loss: 13.8183 - accuracy: 0.5545 - val_loss: 0.6893 - val_accuracy: 0.5588
Epoch 5/5
770/770 [==============================] - 68s 88ms/step - loss: 1.3702 - accuracy: 0.5481 - val_loss: 11.1439 - val_accuracy: 0.5588


## Test the Model
We will now test model's performance with the test data.
- We predict the class for each of the 1155 test using the model.
- We will check the test accuracy.

In [71]:
test_loss, test_acc = model_cnn.evaluate(test_images, test_labels)

1155/1155 [==============================] - 20s 17ms/step


In [72]:
print('test accuracy:', (test_acc*100))

test accuracy: 55.23809790611267
